In [1]:
import tensorflow as tf
from tensorflow.python.keras.testing_utils import layer_test

import numpy as np

W0211 17:23:38.296061 139691842963264 tpu_cluster_resolver.py:35] Falling back to tensorflow client, its recommended to install the cloud tpu client directly with pip install cloud-tpu-client .


## Layer define

### define the layer

In [2]:
@tf.keras.utils.register_keras_serializable(package='dummy-package')
class DummyLayer(tf.keras.layers.Layer):
    # for each tensor, increase value i for each
    def __init__(self, i=1, trainable=True, name=None, dtype=None, dynamic=False, **kwargs):
        self.i = i
        super().__init__(trainable=trainable, name=name, dtype=dtype, dynamic=dynamic, **kwargs)

    def build(self, input_shape):
        self.i_constant = tf.constant(self.i, tf.float32)

        super().build(input_shape)

    def call(self, inputs, **kwargs):
        output = tf.add(inputs, self.i_constant)
        return output

    def get_config(self):
        custom_config = {
            "i": self.i
        }

        base_config = super().get_config()

        config = dict(list(base_config.items()) + list(custom_config.items()))

        return config

### test the layer, make sure it works

In [3]:
layer_test(
    DummyLayer,
    kwargs={"i": 5},
    input_data=np.array([1, 2, 3], np.float32),
    expected_output=np.array([6, 7, 8], np.float32),
    expected_output_dtype=tf.float32,
    validate_training=True,
)

W0211 17:23:39.144376 139691842963264 training_eager.py:274] The list of trainable weights is empty. Make sure that you are not setting model.trainable to False before compiling the model.


array([6., 7., 8.], dtype=float32)

## Save and Load

In [4]:
input_data = np.array([1, 2, 3], np.float32)
expected_output = np.array([6, 7, 8], np.float32)

x = tf.keras.layers.Input(shape=input_data.shape[1:], dtype=input_data.dtype)

layer = DummyLayer(i=5, name="dummy")

model = tf.keras.models.Model(x, layer(x))
model.compile('rmsprop', 'mse')

In [5]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None,)]                 0         
_________________________________________________________________
dummy (DummyLayer)           (None,)                   0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


### save model

In [6]:
model.save("model.h5")

### load and test it

In [7]:
new_model = tf.keras.models.load_model("model.h5")
assert new_model.predict(np.array([1], np.float32)) == np.array([6], np.float32)

### load and test it with custom_object

In [8]:
# NOTE: i is set to 2, not the default value 1 or 5 saved in .h5 file
layer = DummyLayer(i=2, name="dummy")

new_model = tf.keras.models.load_model("model.h5", custom_objects={"dummy": layer})

expected = new_model.predict(np.array([1], np.float32))
print(expected)
assert expected == np.array([3], np.float32), "oops, custom layer instance is not used by model"

[6.]


AssertionError: oops, custom layer instance is not used by model

### remove model file

In [9]:
!rm model.h5